In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

```
pip install yacs
```


```
pip install opencv-python
```

```
conda install scipy
conda install -c conda-forge opencv 
conda install -c conda-forge json_tricks
conda install -c conda-forge pycocotools
```

OSX
```
conda install pytorch torchvision -c pytorch
```

In [2]:
from tools import init_paths
# set up paths


In [3]:
from config import cfg
from config import update_config
from core.loss import JointsMSELoss
from core.function import validate
from utils.utils import create_logger

In [4]:
# cudnn related setting
import torch.backends.cudnn as cudnn
cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED


NameError: name 'torch' is not defined

## load config
```
python test.py --cfg experiments/mpii/hrnet/w32_256x256_adam_lr1e-3.yaml TEST.MODEL_FILE /mnt/models/HRNet/pose_mpii/pose_hrnet_w32_256x256.pth
```

In [5]:
# load config
cfg.defrost()
cfg.merge_from_file('experiments/mpii/hrnet/w32_256x256_adam_lr1e-3.yaml')
cfg.TEST.MODEL_FILE = '/mnt/models/HRNet/pose_mpii/pose_hrnet_w32_256x256.pth'    

In [6]:
cfg.MODEL

CfgNode({'NAME': 'pose_hrnet', 'INIT_WEIGHTS': True, 'PRETRAINED': 'models/pytorch/imagenet/hrnet_w32-36af842e.pth', 'NUM_JOINTS': 16, 'TAG_PER_JOINT': True, 'TARGET_TYPE': 'gaussian', 'IMAGE_SIZE': [256, 256], 'HEATMAP_SIZE': [64, 64], 'SIGMA': 2, 'EXTRA': CfgNode({'PRETRAINED_LAYERS': ['conv1', 'bn1', 'conv2', 'bn2', 'layer1', 'transition1', 'stage2', 'transition2', 'stage3', 'transition3', 'stage4'], 'FINAL_CONV_KERNEL': 1, 'STAGE2': CfgNode({'NUM_MODULES': 1, 'NUM_BRANCHES': 2, 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4], 'NUM_CHANNELS': [32, 64], 'FUSE_METHOD': 'SUM'}), 'STAGE3': CfgNode({'NUM_MODULES': 4, 'NUM_BRANCHES': 3, 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4, 4], 'NUM_CHANNELS': [32, 64, 128], 'FUSE_METHOD': 'SUM'}), 'STAGE4': CfgNode({'NUM_MODULES': 3, 'NUM_BRANCHES': 4, 'BLOCK': 'BASIC', 'NUM_BLOCKS': [4, 4, 4, 4], 'NUM_CHANNELS': [32, 64, 128, 256], 'FUSE_METHOD': 'SUM'})})})

In [7]:
cfg.MODEL.NAME

'pose_hrnet'

In [8]:
from models.pose_hrnet import get_pose_net
model = get_pose_net(cfg,is_train=False)

In [9]:
try:
    logger.info('=> loading model from {}'.format(cfg.TEST.MODEL_FILE))
    model.load_state_dict(torch.load(cfg.TEST.MODEL_FILE), strict=False)
except:
    pass

In [10]:
import torch
model = torch.nn.DataParallel(model)
#model = torch.nn.DataParallel(model, device_ids=cfg.GPUS).cuda()

In [11]:
# define loss function (criterion) and optimizer
criterion = JointsMSELoss(
    use_target_weight=cfg.LOSS.USE_TARGET_WEIGHT
).cuda()

In [12]:
# Data loading code
from torchvision import transforms
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

print(cfg.DATASET.DATASET)
from dataset import mpii
valid_dataset = mpii(
# valid_dataset = eval('dataset.'+cfg.DATASET.DATASET)(
    cfg, cfg.DATASET.ROOT, cfg.DATASET.TEST_SET, False,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=cfg.TEST.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=False,
    num_workers=cfg.WORKERS,
    pin_memory=True
)

mpii


FileNotFoundError: [Errno 2] No such file or directory: 'data/mpii/annot/valid.json'

In [ ]:
# evaluate on validation set
validate(cfg, valid_loader, valid_dataset, model, criterion,
         final_output_dir, tb_log_dir)